## Text Classification


# Load the dataset

In [ ]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


## Train - Test Split

In [ ]:
# Split the training dataset into training and validation sets
train_test_split = dataset["train"].train_test_split(test_size=0.1)
# This splits the training data into 90% training and 10% validation

# Update the dataset dictionary to include the new splits
dataset["train"] = train_test_split["train"]
dataset["validation"] = train_test_split["test"]

# Now, dataset includes three splits: train, validation, and test
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 22500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2500
    })
})


# Data Pre - Processing

In [ ]:
!pip install transformers datasets


In [ ]:
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Tokenize the Dataset

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply the tokenizer to each split
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

## Create data loaders for the training, validation, and test sets.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")


In [ ]:
from torch.utils.data import DataLoader

train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

# Define your batch size
batch_size = 16

# Create the data loaders
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


## Model Training

 Feature Extraction

In [ ]:
# Assuming the datasets are loaded and structured with text and label fields
train_texts = [example['text'] for example in train_dataset]
train_labels = [example['label'] for example in train_dataset]

test_texts = [example['text'] for example in test_dataset]
test_labels = [example['label'] for example in test_dataset]



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer with a maximum of 10,000 features
tfidf_vectorizer = TfidfVectorizer(max_features=10000)

# Fit the vectorizer on the training texts and transform the texts into TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)

# Transform the test texts into TF-IDF vectors based on the learned vocabulary
X_test_tfidf = tfidf_vectorizer.transform(test_texts)


Train the Model

In [ ]:
from sklearn.linear_model import LogisticRegression

# Assuming `train_labels` and `test_labels` contain the sentiment labels
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, train_labels)


LogisticRegression(max_iter=1000)

Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Assuming you have a trained model named `model` and a test dataset `X_test`
# Generate predictions
predictions = model.predict(X_test_tfidf)

# Now assuming `test_labels` contains the true labels for your test dataset
# Compute metrics
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions, average='binary')  # Adjust average as needed
recall = recall_score(test_labels, predictions, average='binary')  # Adjust average as needed

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


Accuracy: 0.882
Precision: 0.8827962161295495
Recall: 0.88096


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from joblib import dump

# Create a pipeline that first vectorizes the text and then applies a logistic regression model
pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression())

# Train the pipeline on your data
pipeline.fit(train_texts, train_labels)

# Save the entire pipeline
dump(pipeline, 'model_with_vectorizer.joblib')


['model_with_vectorizer.joblib']

## Save the Model

In [ ]:
!pip install joblib


In [ ]:
from google.colab import files

files.download('model_with_vectorizer.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>